<a href="https://colab.research.google.com/github/APre203/MLTopic1/blob/main/ML_Topic1_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing All The Requirements

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.4 MB/s eta 0:00:00


In [2]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [3]:
pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.1 MB/s eta 0:00:00


In [4]:
from transformers import pipeline
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import evaluate #from hugging face

In [5]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the destination path
destination_path = '/content/drive/My Drive/new_transcribe.txt'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Map Data

In [6]:
import pandas as pd
data_frames = []
tsv_files = ["/content/drive/MyDrive/cv-corpus-13.0-delta-2023-03-09/en/other.tsv","/content/drive/MyDrive/cv-corpus-13.0-delta-2023-03-09/en/validated.tsv", "/content/drive/MyDrive/cv-corpus-13.0-delta-2023-03-09/en/invalidated.tsv"]
combined_df = pd.DataFrame()
for tsv_file in tsv_files:
    df = pd.read_csv(tsv_file, sep='\t')
    data_frames.append(df)
for d in data_frames:
  combined_df = pd.concat([d, combined_df], axis=0)

mapper = {row["path"]: row for index, row in combined_df.iterrows()}

# print(mapper["common_voice_en_36530278.mp3"])

### Import Model

In [7]:
speech_recognizer = pipeline("automatic-speech-recognition", model="openai/whisper-large-v2")

### Functions for Transcribing and Calculating Cosine Similarity / Word Error Rate

In [8]:
def transcriber(filename): # transcribe audio given the filepath of an audio
    transcribe = speech_recognizer(filename, max_new_tokens=8000)
    return transcribe['text']

In [9]:
def multiple_transcriber(filename, numb, type): # transcribe multiple files given filepath of main file

    audio_file_folder = filename

    all_files = os.listdir(audio_file_folder)

    all_files.sort() #newest to oldest

    overall_cine = 0
    overall_word_error = 0

    audio_files = [file for file in all_files if file.endswith(type)][:numb]
    with open(destination_path, 'w') as file:

      for audio_file in audio_files:
          audio_path = os.path.join(audio_file_folder, audio_file)
          transcription = transcriber(audio_path)
          reference_text = mapper[audio_file]["sentence"]
          overall_cine += csine(transcription,reference_text)
          overall_word_error += word_error(transcription,reference_text)
          # print(transcription, reference_text)
          file.write(transcription + "\n")
    file.close()
    return [overall_cine / numb, overall_word_error / numb]


In [10]:
def csine(text, reference_text):
    # cosine similarity
    txt = [reference_text, text]
    vct = CountVectorizer()
    vector = vct.fit_transform(txt)

    cosine_sim = cosine_similarity(vector)

    similarity = cosine_sim[0][1]

    return similarity

def word_error(text, reference_text):
    wer = evaluate.load("wer")
    return wer.compute(references=[reference_text], predictions=[text])

### Overall Score from the first 100 audios:
### Cosine Similarity: **86.86**%
### Word Error: **20.19**%

In [11]:
filepath = r"/content/drive/MyDrive/cv-corpus-13.0-delta-2023-03-09/en/clips"
multiple_transcriber(filepath, 100, ".mp3")

[0.8686059376908954, 0.20191885891885886]